## Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from math import ceil
import scipy

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K

%matplotlib inline  

# Defining Paths

In [2]:
path = os.getcwd()
img_width, img_height = 224, 224

train_data_dir = os.path.join(path, r'..\Data\train')
val_data_dir = os.path.join(path, r'..\Data\validation')
test_data_dir = os.path.join(path, r'..\Data\test')

batch_size = 8

# Creating Data Generators

In [3]:
train_datagen = ImageDataGenerator(rescale= 1./255, validation_split=0.2, 
                                   rotation_range=20, zoom_range=0.1, 
                                   width_shift_range=0.2, height_shift_range=0.2, 
                                   shear_range=0.1, horizontal_flip=True, fill_mode="nearest")

val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
   train_data_dir,
   target_size=(img_height, img_width),
   batch_size=batch_size,
   class_mode='categorical')

val_generator = test_datagen.flow_from_directory(
   val_data_dir,
   target_size=(img_height, img_width),
   batch_size=batch_size,
   class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
   test_data_dir,
   target_size=(img_height, img_width),
   batch_size=batch_size,
   class_mode='categorical')

Found 233 images belonging to 3 classes.
Found 55 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


# Load and compile pretrained Models

## Learning rate scheduler

In [4]:
##https://towardsdatascience.com/learning-rate-schedule-in-practice-an-example-with-keras-and-tensorflow-2-0-2f48b2888a0c
initial_learning_rate = 0.001
epochs = 100
decay = initial_learning_rate / epochs

def lr_schedule(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

In [5]:
#create a Callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=True)

## ResNET-50

In [8]:
model_res50 = applications.ResNet50(weights='imagenet', include_top=False) 
# add a global spatial average pooling layer
x = model_res50.output
x = GlobalAveragePooling2D()(x)

# add a flatten layer
x = Flatten()(x)
# add a dropout layer
x = Dropout(0.2)(x)

# and a fully connected output/classification layer
predictions = Dense(3, activation='softmax')(x)

# create the full network
model_transfer = Model(inputs=model_res50.input, outputs=predictions)

In [9]:
# compile
model_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(),
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
history_pretrained = model_transfer.fit(train_generator, epochs=epochs, shuffle = True, verbose = 1, validation_data = val_generator, callbacks=[lr_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/100
30/30 [==============================] - 63s 2s/step - loss: 1.2184 - accuracy: 0.4850 - recall: 0.4378 - precision: 0.5000 - val_loss: 1.5407 - val_accuracy: 0.3636 - val_recall: 0.3636 - val_precision: 0.3636 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999899777484807.
Epoch 2/100
30/30 [==============================] - 59s 2s/step - loss: 0.6949 - accuracy: 0.6953 - recall: 0.6652 - precision: 0.7209 - val_loss: 1.9567 - val_accuracy: 0.3636 - val_recall: 0.3636 - val_precision: 0.3636 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999700130964479.
Epoch 3/100
30/30 [==============================] - 61s 2s/step - loss: 0.4718 - accuracy: 0.7983 - recall: 0.7854 - precision: 0.8133 - val_loss: 1.2640 - val_accuracy: 0.3636 - val_recall: 0.3636 - val_precision: 0.3636 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate t

In [ ]:
# summarize history for accuracy
plt.plot(history_pretrained.history['val_accuracy'], "-b", label="validation_acc")
plt.plot(history_pretrained.history['accuracy'], "-r", label="train_acc")
plt.title('Resnet-50 Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

# summarize history for loss
plt.plot(history_pretrained.history['val_loss'], "-b", label="validation_loss")
plt.plot(history_pretrained.history['loss'], "-r", label="train_loss")
plt.title('Resnet-50 Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

# summarize history for precision
plt.plot(history_pretrained.history['val_precision'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.show()

# summarize history for recall
plt.plot(history_pretrained.history['val_recall'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.show()

In [ ]:
model_transfer.evaluate(test_generator)

## VGG-16

In [ ]:
model_vgg16 = applications.VGG16(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = model_vgg16.output
x = GlobalAveragePooling2D()(x)

# add a flatten layer
x = Flatten()(x)
# add a dropout layer
x = Dropout(0.2)(x)

# and a fully connected output/classification layer
predictions = Dense(3, activation='softmax')(x)

# create the full network
model_transfer_vgg = Model(inputs=model_vgg16.input, outputs=predictions)

In [ ]:
# compile
model_transfer_vgg.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(),
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
history_pretrained_vgg = model_transfer_vgg.fit(train_generator, epochs=epochs, shuffle = True, verbose = 1, validation_data = val_generator, , callbacks=[lr_callback])

In [ ]:
# summarize history for accuracy
plt.plot(history_pretrained_vgg.history['val_accuracy'], "-b", label="validation_acc")
plt.plot(history_pretrained_vgg.history['accuracy'], "-r", label="train_acc")
plt.title('VGG-16 Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

# summarize history for loss
plt.plot(history_pretrained_vgg.history['val_loss'], "-b", label="validation_loss")
plt.plot(history_pretrained_vgg.history['loss'], "-r", label="train_loss")
plt.title('VGG-16 Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

# summarize history for precision
plt.plot(history_pretrained_vgg.history['val_precision_1'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.show()

# summarize history for recall
plt.plot(history_pretrained_vgg.history['val_recall_1'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.show()

In [ ]:
model_transfer_vgg.evaluate(test_generator)

# Saving Model

In [ ]:
model_path = './../Trained_Model/model_resnet.h5'
model_transfer.save(model_path)

In [ ]:
model_path_vgg = './../Trained_Model/model_vgg.h5'
model_transfer_vgg.save(model_path_vgg)